In [10]:
!export MLFLOW_TRACKING_URI=http://localhost:5000


In [4]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [12]:
mlflow.set_tracking_uri("http://127.0.0.1:8000")

In [5]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)


In [17]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=np.mean(train_x), variance=np.var(train_x)),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "eval_rmse": eval_rmse, "status": STATUS_OK, "model": model}


In [18]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result


In [8]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}


In [19]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["eval_rmse"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['eval_rmse']}")


Epoch 1/3                                            

46/46 [==============================] - 1s 9ms/step - loss: 4.9628 - root_mean_squared_error: 2.2277 - val_loss: 1.6049 - val_root_mean_squared_error: 1.2668

Epoch 2/3                                            

46/46 [==============================] - 0s 4ms/step - loss: 1.1365 - root_mean_squared_error: 1.0661 - val_loss: 0.8846 - val_root_mean_squared_error: 0.9405

Epoch 3/3                                            

46/46 [==============================] - 0s 3ms/step - loss: 0.8314 - root_mean_squared_error: 0.9118 - val_loss: 0.7939 - val_root_mean_squared_error: 0.8910

12/12 [==============================] - 0s 2ms/step - loss: 0.7939 - root_mean_squared_error: 0.8910

  0%|          | 0/8 [00:01<?, ?trial/s, best loss=?]INFO:tensorflow:Assets written to: /var/folders/pr/2nrw06tn1vb56m13zv7wlrsm0000gn/T/tmp506q5l4z/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/pr/2nrw06tn1vb56m13zv7wlrsm0000gn/T/tmp506q5l4z/model/data/model/assets



Epoch 1/3                                                                     

46/46 [==============================] - 1s 7ms/step - loss: 10.3912 - root_mean_squared_error: 3.2235 - val_loss: 4.0179 - val_root_mean_squared_error: 2.0045

Epoch 2/3                                                                     

46/46 [==============================] - 0s 3ms/step - loss: 3.6048 - root_mean_squared_error: 1.8986 - val_loss: 3.6507 - val_root_mean_squared_error: 1.9107

Epoch 3/3                                                                     

46/46 [==============================] - 0s 2ms/step - loss: 3.3562 - root_mean_squared_error: 1.8320 - val_loss: 3.4031 - val_root_mean_squared_error: 1.8447

12/12 [==============================] - 0s 2ms/step - loss: 3.4031 - root_mean_squared_error: 1.8447

 12%|█▎        | 1/8 [00:11<01:13, 10.46s/trial, best loss: 0.891036331653595]INFO:tensorflow:Assets written to: /var/folders/pr/2nrw06tn1vb56m13zv7wlrsm0000gn/T/tmpvd4shme2/mo

INFO:tensorflow:Assets written to: /var/folders/pr/2nrw06tn1vb56m13zv7wlrsm0000gn/T/tmpvd4shme2/model/data/model/assets



Epoch 1/3                                                                     

46/46 [==============================] - 1s 5ms/step - loss: 21.8522 - root_mean_squared_error: 4.6746 - val_loss: 15.2003 - val_root_mean_squared_error: 3.8988

Epoch 2/3                                                                     

46/46 [==============================] - 0s 3ms/step - loss: 10.4317 - root_mean_squared_error: 3.2298 - val_loss: 7.4427 - val_root_mean_squared_error: 2.7281

Epoch 3/3                                                                     

46/46 [==============================] - 0s 2ms/step - loss: 5.5011 - root_mean_squared_error: 2.3454 - val_loss: 4.7704 - val_root_mean_squared_error: 2.1841

12/12 [==============================] - 0s 2ms/step - loss: 4.7704 - root_mean_squared_error: 2.1841

 25%|██▌       | 2/8 [00:19<00:52,  8.81s/trial, best loss: 0.891036331653595]INFO:tensorflow:Assets written to: /var/folders/pr/2nrw06tn1vb56m13zv7wlrsm0000gn/T/tmpwzq45txv/

...

Epoch 1/3                                                                     

46/46 [==============================] - 1s 6ms/step - loss: 3.7290 - root_mean_squared_error: 1.9311 - val_loss: 0.9453 - val_root_mean_squared_error: 0.9723

Epoch 2/3                                                                     

46/46 [==============================] - 0s 4ms/step - loss: 0.8283 - root_mean_squared_error: 0.9101 - val_loss: 0.7638 - val_root_mean_squared_error: 0.8740

Epoch 3/3                                                                     

46/46 [==============================] - 0s 3ms/step - loss: 0.7859 - root_mean_squared_error: 0.8865 - val_loss: 0.7532 - val_root_mean_squared_error: 0.8679

12/12 [==============================] - 0s 2ms/step - loss: 0.7532 - root_mean_squared_error: 0.8679

 88%|████████▊ | 7/8 [00:59<00:08,  8.07s/trial, best loss: 0.891036331653595]INFO:tensorflow:Assets written to: /var/folders/pr/2nrw06tn1vb56m13zv7wlrsm0000gn/T/tmpw0acy8gv/mod

INFO:tensorflow:Assets written to: /var/folders/pr/2nrw06tn1vb56m13zv7wlrsm0000gn/T/tmpw0acy8gv/model/data/model/assets



100%|██████████| 8/8 [01:06<00:00,  8.34s/trial, best loss: 0.8678948283195496]
INFO:tensorflow:Assets written to: /var/folders/pr/2nrw06tn1vb56m13zv7wlrsm0000gn/T/tmpkv9tmxid/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/pr/2nrw06tn1vb56m13zv7wlrsm0000gn/T/tmpkv9tmxid/model/data/model/assets



Best parameters: {'lr': 0.005150425768294159, 'momentum': 0.788207874627587}
Best eval rmse: 0.8678948283195496
